In [ ]:
# EU HICP Package Holidays Price Forecast - Phase 1: Data Collection & Cleaning

## Overview
This notebook implements **Phase 1** of the EU HICP Package Holidays Price Forecast project, focusing on comprehensive data collection and cleaning using the FRED, BLS, Eurostat, and ECB APIs.

### Objectives
1. **Collect HICP data** for EU and Germany package holidays (CP96EAMM, CP96DEMM)
2. **Gather economic indicators** that influence travel demand
3. **Fetch comparative data** from US travel markets via BLS
4. **Clean and structure data** using polars for efficient processing
5. **Create unified dataset** for subsequent analysis phases

### Data Sources
- **FRED (Federal Reserve Economic Data)**: HICP indices and economic indicators
- **BLS (Bureau of Labor Statistics)**: US travel price indices for comparison
- **Eurostat**: EU tourism statistics (future enhancement)
- **ECB**: European Central Bank monetary indicators (future enhancement)

### Key Technologies
- **Polars**: Fast DataFrame operations and lazy evaluation
- **NumPy**: Numerical computations
- **Plotly**: Interactive visualizations
- **FRED/BLS APIs**: Data collection


In [ ]:
# Import required libraries
import polars as pl
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import sys
import os

# Add project root to path for imports
sys.path.append('.')

# Import project modules
from config import validate_api_keys, HICP_SERIES, ECONOMIC_INDICATORS
from data_collector import DataCollector

# Configure polars
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(10)
pl.Config.set_fmt_str_lengths(50)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")
print(f"Polars version: {pl.__version__}")
print(f"NumPy version: {np.__version__}")

# Validate API configuration
print("\n" + "="*50)
print("API CONFIGURATION CHECK")
print("="*50)
validate_api_keys()


In [ ]:
## Step 1: Initialize Data Collector and Validate Configuration

Before collecting data, we'll initialize our data collector and verify that all API keys are properly configured. The system will work with missing API keys but with limited functionality.


In [ ]:
# Initialize the data collector
collector = DataCollector()

# Display the data series we'll be collecting
print("TARGET HICP SERIES:")
print("="*40)
for name, series_id in HICP_SERIES.items():
    print(f"  {name}: {series_id}")

print("\nECONOMIC INDICATORS:")
print("="*40)
for name, series_id in ECONOMIC_INDICATORS.items():
    print(f"  {name}: {series_id}")

print(f"\nData collection period: 2010-01-01 to 2024-12-31")
print(f"Forecast target: July 2025")


In [ ]:
## Step 2: Collect All Data from APIs

Now we'll collect data from all available sources. This process will:
1. **Fetch FRED data**: HICP indices and economic indicators
2. **Collect BLS data**: US travel price indices for comparative analysis
3. **Save raw data**: Store collected data in parquet format for efficient access
4. **Generate summary**: Provide overview of collected datasets

*Note: If you don't have API keys configured, you can still explore the notebook structure and methods.*


In [ ]:
# Collect all data from APIs
print("Starting comprehensive data collection...")
print("This may take a few minutes depending on API response times.\n")

# Collect all datasets
datasets = collector.collect_all_data()

# Save the collected data
collector.save_data(datasets, format='parquet')

# Also save as CSV for easy inspection
collector.save_data(datasets, format='csv')

print("\n" + "="*60)
print("DATA COLLECTION COMPLETE")
print("="*60)


In [ ]:
## Step 3: Data Quality Assessment and Summary

Let's examine the collected data to understand its structure, coverage, and quality. This analysis will help us identify any data gaps or issues that need to be addressed in the cleaning phase.


In [ ]:
# Generate comprehensive data summary
def analyze_dataset(df: pl.DataFrame, dataset_name: str) -> None:
    """Analyze a dataset and print comprehensive summary."""
    if df.is_empty():
        print(f"\n{dataset_name.upper()} Dataset: EMPTY")
        return
    
    print(f"\n{dataset_name.upper()} Dataset Analysis:")
    print("="*50)
    
    # Basic statistics
    print(f"Rows: {len(df):,}")
    print(f"Columns: {df.width}")
    
    # Date range
    if 'date' in df.columns:
        date_min = df['date'].min()
        date_max = df['date'].max()
        print(f"Date range: {date_min} to {date_max}")
        
        # Calculate data frequency
        unique_dates = df['date'].unique().sort()
        if len(unique_dates) > 1:
            # Estimate frequency based on first few date differences
            date_diffs = []
            for i in range(min(10, len(unique_dates)-1)):
                diff = unique_dates[i+1] - unique_dates[i]
                date_diffs.append(diff.days if hasattr(diff, 'days') else diff)
            
            avg_diff = np.mean(date_diffs) if date_diffs else 0
            if 28 <= avg_diff <= 31:
                freq = "Monthly"
            elif 7 <= avg_diff <= 7:
                freq = "Weekly"
            elif avg_diff == 1:
                freq = "Daily"
            else:
                freq = f"~{avg_diff:.1f} days"
            
            print(f"Estimated frequency: {freq}")
    
    # Series information
    if 'series_name' in df.columns:
        series = df['series_name'].unique().to_list()
        print(f"Number of series: {len(series)}")
        print("Series names:")
        for s in series:
            series_count = df.filter(pl.col('series_name') == s).height
            print(f"  • {s}: {series_count:,} observations")
    
    # Missing data analysis
    if 'value' in df.columns:
        total_obs = len(df)
        missing_obs = df.filter(pl.col('value').is_null()).height
        missing_pct = (missing_obs / total_obs) * 100 if total_obs > 0 else 0
        
        print(f"\nData Quality:")
        print(f"Missing values: {missing_obs:,} ({missing_pct:.1f}%)")
        
        if missing_obs < total_obs:
            value_stats = df.filter(pl.col('value').is_not_null())['value']
            print(f"Value range: {value_stats.min():.2f} to {value_stats.max():.2f}")
            print(f"Mean value: {value_stats.mean():.2f}")
    
    # Display sample data
    print(f"\nSample data (first 5 rows):")
    print(df.head(5))

# Analyze each dataset
for name, df in datasets.items():
    analyze_dataset(df, name)


In [ ]:
## Step 4: Data Cleaning and Standardization

Now we'll clean and standardize the collected data using polars' efficient operations. This includes:

1. **Handle missing values**: Implement appropriate imputation strategies
2. **Standardize formats**: Ensure consistent date and value formats
3. **Create wide format**: Pivot data for easier analysis
4. **Add derived variables**: Calculate month-over-month changes and other features
5. **Validate data integrity**: Check for outliers and inconsistencies


In [ ]:
def clean_and_transform_data(datasets: dict) -> pl.DataFrame:
    """
    Clean and transform collected datasets into unified format.
    
    Args:
        datasets: Dictionary of raw datasets from APIs
        
    Returns:
        Clean, unified polars DataFrame ready for analysis
    """
    
    print("Starting data cleaning and transformation...")
    
    # Combine all datasets
    all_data = []
    for source_name, df in datasets.items():
        if df.is_empty():
            print(f"Skipping empty {source_name} dataset")
            continue
            
        # Add source identifier
        df_with_source = df.with_columns([
            pl.lit(source_name).alias('data_source')
        ])
        all_data.append(df_with_source)
    
    if not all_data:
        print("Warning: No data to clean")
        return pl.DataFrame()
    
    # Combine all datasets
    combined_df = pl.concat(all_data, how='vertical_relaxed')
    print(f"✓ Combined {len(all_data)} datasets into {len(combined_df):,} total observations")
    
    # Data cleaning steps
    print("\nApplying data cleaning steps...")
    
    # 1. Ensure proper data types and sort by date
    cleaned_df = (
        combined_df
        .with_columns([
            pl.col('date').cast(pl.Date),
            pl.col('value').cast(pl.Float64)
        ])
        .sort(['series_name', 'date'])
    )
    
    # 2. Handle missing values (forward fill within each series)
    cleaned_df = (
        cleaned_df
        .with_columns([
            pl.col('value').forward_fill().over('series_name').alias('value_filled')
        ])
    )
    
    # 3. Calculate month-over-month percentage changes
    cleaned_df = (
        cleaned_df
        .with_columns([
            # MoM percentage change
            (
                (pl.col('value_filled') / pl.col('value_filled').shift(1).over('series_name') - 1) * 100
            ).alias('mom_pct_change'),
            
            # Year-over-year percentage change
            (
                (pl.col('value_filled') / pl.col('value_filled').shift(12).over('series_name') - 1) * 100
            ).alias('yoy_pct_change')
        ])
    )
    
    # 4. Add temporal features
    cleaned_df = (
        cleaned_df
        .with_columns([
            pl.col('date').dt.year().alias('year'),
            pl.col('date').dt.month().alias('month'),
            pl.col('date').dt.quarter().alias('quarter'),
            
            # Seasonal indicators
            pl.when(pl.col('date').dt.month().is_in([6, 7, 8]))
            .then(pl.lit('Summer'))
            .when(pl.col('date').dt.month().is_in([12, 1, 2]))
            .then(pl.lit('Winter'))
            .when(pl.col('date').dt.month().is_in([3, 4, 5]))
            .then(pl.lit('Spring'))
            .otherwise(pl.lit('Autumn'))
            .alias('season'),
            
            # Holiday season indicator
            pl.when(pl.col('date').dt.month().is_in([6, 7, 8]))
            .then(pl.lit(True))
            .otherwise(pl.lit(False))
            .alias('is_holiday_season')
        ])
    )
    
    print("✓ Applied forward fill for missing values")
    print("✓ Calculated MoM and YoY percentage changes")
    print("✓ Added temporal and seasonal features")
    
    # 5. Data quality checks
    print("\nData quality assessment:")
    
    # Check for extreme outliers (beyond 3 standard deviations)
    outlier_threshold = 3
    for series in cleaned_df['series_name'].unique():
        series_data = cleaned_df.filter(pl.col('series_name') == series)['mom_pct_change']
        series_data_clean = series_data.drop_nulls()
        
        if len(series_data_clean) > 0:
            mean_val = series_data_clean.mean()
            std_val = series_data_clean.std()
            
            if std_val and std_val > 0:
                outliers = series_data_clean.filter(
                    (series_data_clean - mean_val).abs() > outlier_threshold * std_val
                )
                
                if len(outliers) > 0:
                    print(f"  ⚠️  {series}: {len(outliers)} potential outliers detected")
                else:
                    print(f"  ✓ {series}: No extreme outliers detected")
    
    return cleaned_df

# Apply cleaning and transformation
clean_data = clean_and_transform_data(datasets)

# Display cleaning results
if not clean_data.is_empty():
    print(f"\n" + "="*60)
    print("CLEANING RESULTS")
    print("="*60)
    print(f"Final dataset shape: {clean_data.height:,} rows × {clean_data.width} columns")
    print(f"Date range: {clean_data['date'].min()} to {clean_data['date'].max()}")
    print(f"Series included: {clean_data['series_name'].n_unique()}")
    
    # Missing data summary
    missing_values = clean_data['value'].null_count()
    missing_filled = clean_data['value_filled'].null_count()
    print(f"Missing values (original): {missing_values:,}")
    print(f"Missing values (after cleaning): {missing_filled:,}")
else:
    print("❌ No data available for cleaning")


In [ ]:
## Step 5: Create Wide Format Dataset for Analysis

For easier analysis and modeling, we'll create a wide format version of our data where each series becomes a column. This will facilitate cross-series correlations and feature engineering.


In [ ]:
def create_wide_format_data(df: pl.DataFrame) -> pl.DataFrame:
    """
    Convert long format data to wide format for analysis.
    
    Args:
        df: Clean long format DataFrame
        
    Returns:
        Wide format DataFrame with series as columns
    """
    
    if df.is_empty():
        print("Cannot create wide format - no data available")
        return pl.DataFrame()
    
    print("Creating wide format dataset...")
    
    # Select key columns and pivot
    wide_values = (
        df
        .select(['date', 'series_name', 'value_filled', 'year', 'month', 'quarter', 'season', 'is_holiday_season'])
        .unique(['date', 'series_name'])  # Remove any duplicates
        .pivot(
            values='value_filled',
            index=['date', 'year', 'month', 'quarter', 'season', 'is_holiday_season'],
            columns='series_name'
        )
    )
    
    # Create wide format for MoM changes
    wide_mom_base = (
        df
        .select(['date', 'series_name', 'mom_pct_change'])
        .unique(['date', 'series_name'])
        .pivot(
            values='mom_pct_change',
            index='date',
            columns='series_name'
        )
    )
    wide_mom = wide_mom_base.rename({col: f"{col}_mom_pct" for col in wide_mom_base.columns if col != 'date'})
    
    # Create wide format for YoY changes  
    wide_yoy_base = (
        df
        .select(['date', 'series_name', 'yoy_pct_change'])
        .unique(['date', 'series_name'])
        .pivot(
            values='yoy_pct_change',
            index='date',
            columns='series_name'
        )
    )
    wide_yoy = wide_yoy_base.rename({col: f"{col}_yoy_pct" for col in wide_yoy_base.columns if col != 'date'})
    
    # Join all wide format datasets
    wide_final = (
        wide_values
        .join(wide_mom, on='date', how='left')
        .join(wide_yoy, on='date', how='left')
        .sort('date')
    )
    
    print(f"✓ Created wide format dataset: {wide_final.height:,} rows × {wide_final.width} columns")
    
    return wide_final

# Create wide format data
if not clean_data.is_empty():
    wide_data = create_wide_format_data(clean_data)
    
    # Display sample of wide format data
    if not wide_data.is_empty():
        print("\nSample of wide format data:")
        print(wide_data.head(10))
        
        # Summary of columns
        print(f"\nColumns in wide format dataset:")
        for i, col in enumerate(wide_data.columns):
            print(f"  {i+1:2d}. {col}")
    
else:
    print("Cannot create wide format - no clean data available")
    wide_data = pl.DataFrame()


In [ ]:
## Step 6: Save Cleaned Data and Summary

Let's save our cleaned and processed data for use in subsequent analysis phases and create a final summary of what we've accomplished.


In [ ]:
# Save cleaned datasets
def save_cleaned_data():
    """Save cleaned data in multiple formats for different use cases."""
    
    print("Saving cleaned and processed data...")
    
    if not clean_data.is_empty():
        # Save long format (for time series analysis)
        clean_data.write_parquet('data/clean_long_format.parquet')
        clean_data.write_csv('data/clean_long_format.csv')
        print("✓ Saved long format data")
    
    if not wide_data.is_empty():
        # Save wide format (for cross-sectional analysis)
        wide_data.write_parquet('data/clean_wide_format.parquet')
        wide_data.write_csv('data/clean_wide_format.csv')
        print("✓ Saved wide format data")
    
    # Create metadata file
    metadata = {
        'creation_date': datetime.now().isoformat(),
        'data_sources': list(datasets.keys()) if datasets else [],
        'series_collected': clean_data['series_name'].unique().to_list() if not clean_data.is_empty() else [],
        'date_range': {
            'start': str(clean_data['date'].min()) if not clean_data.is_empty() else None,
            'end': str(clean_data['date'].max()) if not clean_data.is_empty() else None
        },
        'total_observations': len(clean_data) if not clean_data.is_empty() else 0,
        'columns_wide_format': wide_data.columns if not wide_data.is_empty() else []
    }
    
    import json
    with open('data/data_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print("✓ Saved metadata file")

save_cleaned_data()

# Final summary
print("\n" + "="*70)
print("PHASE 1 COMPLETION SUMMARY")
print("="*70)

if not clean_data.is_empty():
    print(f"✓ Successfully collected and cleaned data from {len(datasets)} sources")
    print(f"✓ {len(clean_data):,} total observations across {clean_data['series_name'].n_unique()} series")
    print(f"✓ Date coverage: {clean_data['date'].min()} to {clean_data['date'].max()}")
    print(f"✓ Added {clean_data.width - 4} derived features (MoM%, YoY%, temporal features)")
    print(f"✓ Created wide format dataset with {wide_data.width} columns")
    
    # Key series summary
    print(f"\nKey HICP Series Status:")
    for series_name in ['eu_package_holidays', 'germany_package_holidays']:
        series_data = clean_data.filter(pl.col('series_name') == series_name)
        if not series_data.is_empty():
            print(f"  ✓ {series_name}: {len(series_data)} observations")
        else:
            print(f"  ❌ {series_name}: No data collected")
    
    print(f"\nFiles Created:")
    print(f"  • data/clean_long_format.parquet (primary analysis dataset)")
    print(f"  • data/clean_wide_format.parquet (cross-sectional analysis)")
    print(f"  • data/clean_long_format.csv (human-readable format)")
    print(f"  • data/clean_wide_format.csv (human-readable format)")
    print(f"  • data/data_metadata.json (dataset documentation)")
    
else:
    print("❌ Phase 1 completed with limited success - no data was collected")
    print("   This may be due to missing API keys or connectivity issues")
    print("   Please check your API configuration and try again")

print(f"\n🎯 Ready for Phase 2: Exploratory Data Analysis")
print(f"   Next notebook: 02_exploratory_data_analysis.ipynb")
